# Phase 1: Classical ML (regression)
Focus on data understanding and traditional models

## Step 1. Data Processing

Data source: https://insideairbnb.com/get-the-data/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 1. Load raw data
url = "https://data.insideairbnb.com/thailand/central-thailand/bangkok/2025-09-26/data/listings.csv.gz"
df_list = pd.read_csv(url, compression="gzip")
#df_list.head(2)

In [ ]:
df_list.columns

In [ ]:
df_list.info()

In [ ]:
df_list.iloc[2].to_dict()

In [ ]:
# 2. CLEAN PRICE COLUMN
# drop na
df_list = df_list.dropna(subset=['price'])
df_list.info() # 23273 entries

In [ ]:
# convert price to numeric

df_list['price'] = (df_list['price'].astype(str).str.replace(r'[^0-9.]', '', regex=True).replace('', np.nan).astype(float))

In [ ]:
print(df_list['price'].describe().to_string())

In [ ]:
# Plot prices
prices = df_list['price']
prices = prices[prices > 0]
plt.figure(figsize=(15, 6))
plt.title(f"Price Distribution: Avg {prices.mean():,.2f} THB/night | Max {prices.max():,} THB\n")
plt.xlabel("Price (THB)")
plt.ylabel("Count")

plt.hist(prices, rwidth=0.7, color="orange", bins=range(0, 100000, 50))

plt.show()

In [ ]:
# Find the maximum price
max_price = df_list['price'].max()
#print("Maximum price:", max_price)

# Get the row(s) with that price
max_price_rows = df_list[df_list['price'] == max_price]
max_price_rows

In [ ]:
max_price_rows.iloc[1].to_dict()

In [ ]:
# Filter extreme prices
# remove prices <=0 and extreme outliers 
print(f"mean: {df_list['price'].mean()}")
print(f"median: {df_list['price'].median()}")
print(f"max: {df_list['price'].max()}")
print(f"min: {df_list['price'].min()}")

In [14]:
# keep price between 100 and 20,000 THB (reasonable range for nightly rentals)

df_list = df_list[(df_list['price'] > 0) & 
                  (df_list['price'] >= 100) & 
                  (df_list['price'] <= 20000)].copy()

In [ ]:
print(f"mean: {df_list['price'].mean()}")
print(f"median: {df_list['price'].median()}")
print(f"max: {df_list['price'].max()}")
print(f"min: {df_list['price'].min()}")

In [ ]:
# Plot prices AFTER filtering
prices = df_list['price']
prices = prices[prices > 0]

plt.figure(figsize=(15, 6))
plt.title(f"Price Distribution (Filtered): Avg {prices.mean():,.2f} THB/night | Max {prices.max():,} THB\n")
plt.xlabel("Price (THB)")
plt.ylabel("Count")

plt.hist(prices, rwidth=0.7, color="orange", bins=50)

plt.show()

In [17]:
# load review data
url = "https://data.insideairbnb.com/thailand/central-thailand/bangkok/2025-09-26/data/reviews.csv.gz"
df_review = pd.read_csv(url, compression="gzip")
# df_review.head()

In [ ]:
df_review.head()

In [18]:
# long to wide convert
# 3. AGGREGATE REVIEWS → ONE TEXT PER LISTING
grouped = df_review.groupby('listing_id')['comments'].apply(lambda texts: " ".join(texts.dropna().astype(str))).reset_index()
grouped.rename(columns={'comments':'all_review_texts'}, inplace=True)

# Merge listings + aggregated reviews
df = df_list.merge(grouped, how='left', left_on='id', right_on='listing_id')

In [ ]:
df.iloc[10].to_dict()

In [ ]:
df.info()

In [19]:
#-----------------------------------------------------------------------------------------------
# TEXT CLEANING
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)

    # Remove <br>, <br/>, <BR>, <br /> etc.
    text = re.sub(r'<\s*br\s*/?\s*>', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'<[^>]+>', '', text)  # Remove other HTML tags

    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)

    # Remove emojis and non-ASCII characters
    text = text.encode('ascii', 'ignore').decode()

    # Clean special characters but keep basic punctuation
    text = re.sub(r'[:\[\]"{}""'']+', ' ', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Clean up comma issues
    text = text.replace(" ,", ",").replace(",,,", ",").replace(",,", ",")

    return text

In [20]:
# Clean text fields
for col in ["name", "description", "neighborhood_overview","property_type","room_type", "host_about", "bathrooms_text","host_location","all_review_texts"]:
    df[col] = df[col].astype(str).apply(clean_text)

In [ ]:
df.iloc[1].to_dict()

In [21]:
# process amenitis
from ast import literal_eval

def parse_amenities(a):
    try:
        lst = literal_eval(a)
        return ", ".join(lst)
    except:
        return ""

df["amenities_clean"] = df["amenities"].apply(parse_amenities)

In [ ]:
df.iloc[10].to_dict()

## Step 2: Feature Engineering

In [ ]:
# Derived features from reviews and host
df.info()

In [22]:
# review text length (proxy for reviewer activity/content)
df['all_reviews_len'] = df['all_review_texts'].fillna("").apply(len)

In [23]:
df['num_amenities'] = df['amenities'].fillna("[]").apply(lambda x: len(eval(x)) if isinstance(x, str) and x.strip().startswith('[') else 0)

In [24]:
from datetime import datetime

def parse_date_safe(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d")
    except:
        return np.nan

df['host_since_dt'] = df['host_since'].apply(parse_date_safe)
now = pd.to_datetime(df['last_scraped'], errors='coerce').fillna(pd.to_datetime("2025-09-27"))
df['host_tenure_years'] = (now - df['host_since_dt']).dt.days / 365

In [25]:
# simple binary conversions
df['host_is_superhost_bin'] = df['host_is_superhost'].map({'t':1,'f':0})
df['instant_bookable_bin'] = df['instant_bookable'].map({'t':1,'f':0})

In [26]:
# fill missing numeric counts with 0 where logical
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [ ]:
# 3b) Handle obvious missingness for numeric columns
# We'll let the pipeline impute medians, but check distributions
print(df[['bedrooms','beds','bathrooms']].describe())

In [ ]:
# Keep a sane set of candidate features
candidate_features = [
    'room_type', 'property_type', 'neighbourhood_cleansed',
    'accommodates', 'bedrooms', 'beds', 'bathrooms',
    'num_amenities', 'all_reviews_len', 'number_of_reviews', 'reviews_per_month',
    'review_scores_rating', 'host_tenure_years', 'host_is_superhost_bin', 'instant_bookable_bin',
    'latitude', 'longitude', 'minimum_nights'
]

# Quick check
df[candidate_features + ['price']].info()

In [28]:
# 4) Prepare target: log-transform price because of strong right skew
df = df[df['price'] > 0].copy()  # ensure positive
df['log_price'] = np.log1p(df['price'])   # target for regression

In [29]:
from sklearn.model_selection import train_test_split, cross_val_score

# For reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Split into train/test
feature_df = df[candidate_features].copy()
target = df['log_price'].copy()

X_train, X_test, y_train, y_test = train_test_split(feature_df, target, test_size=0.20, random_state=RANDOM_STATE)

print("Train size:", X_train.shape, "Test size:", X_test.shape)

Train size: (18473, 18) Test size: (4619, 18)


In [ ]:
df.iloc[10].to_dict()

In [30]:
# 5) Build preprocessing pipelines
numeric_features = [
    'accommodates','bedrooms','beds','bathrooms',
    'num_amenities','all_reviews_len','number_of_reviews','reviews_per_month',
    'review_scores_rating','host_tenure_years','latitude','longitude','minimum_nights'
]
# categorical features we'll one-hot
categorical_features = ['room_type','property_type','neighbourhood_cleansed']

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# numeric transformer: impute median, then scale
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# categorical transformer: impute 'missing' then OneHot
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [32]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
], remainder='drop')

# full pipeline: preprocess -> linear regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('reg', LinearRegression())
])

In [33]:
# 6) Fit model
pipeline.fit(X_train, y_train)

# Predictions in log-space
y_pred_log = pipeline.predict(X_test)


In [34]:
# Convert back to original price
y_pred = np.expm1(y_pred_log)  # inverse of log1p
y_true = np.expm1(y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²: {r2:.4f}")

Define Color Coding for Errors:
- Very Good → Green
- Good → Yellow
- So-so / Bad → Red

In [36]:
# Color map for terminal printing
COLOR_MAP = {
    "green": "\033[92m",    # good
    "orange": "\033[93m",   # okay
    "red": "\033[91m"       # bad
}
RESET = "\033[0m"

def color_for_error(pred, truth):
    error_ratio = abs(pred - truth) / (truth + 1e-6)

    if error_ratio < 0.10:
        return "green"   # <10% error → very good
    elif error_ratio < 0.30:
        return "orange"  # <30% error → acceptable
    else:
        return "red"     # >30% error → bad

In [ ]:
import math

records = []

print("\n======== PREDICTION REPORT ========\n")

for i, (pred, truth) in enumerate(zip(y_pred, y_true)):
    error = abs(pred - truth)
    sle = (math.log1p(truth) - math.log1p(pred)) ** 2
    color = color_for_error(pred, truth)

    records.append({
        "index": i,
        "truth": truth,
        "pred": pred,
        "error": error,
        "sle": sle,
        "ape": error / truth if truth != 0 else np.nan,   # absolute percentage error
        "hit": 1 if truth != 0 and (error / truth) <= 0.2 else 0,
        "color": color
    })

    print(
        f"{COLOR_MAP[color]}"
        f"{i+1:03d}: "
        f"Pred: {pred:,.0f} THB | "
        f"Truth: {truth:,.0f} THB | "
        f"Error: {error:,.0f} | "
        f"SLE: {sle:.4f}"
        f"{RESET}"
    )

In [38]:
eval_df = pd.DataFrame(records)

In [ ]:
# ===== METRICS =====
mae = eval_df["error"].mean()
mape = eval_df["ape"].mean() * 100
rmsle = np.sqrt(eval_df["sle"].mean())
hit_rate = eval_df["hit"].mean() * 100

print("\n======== METRICS ========")
print(f"MAE:   {mae:,.2f} THB")
print(f"MAPE:  {mape:.2f}%")
print(f"RMSLE: {rmsle:.4f}")
print(f"Hit-rate (<=20% error): {hit_rate:.2f}%")
print("==========================\n")

In [ ]:
plt.figure(figsize=(12, 8))

max_val = max(max(y_pred), max(y_true))
plt.plot([0, max_val], [0, max_val], lw=2, alpha=0.5)

plt.scatter(
    eval_df["truth"],
    eval_df["pred"],
    c=eval_df["color"].map({
        "green": "green",
        "orange": "orange",
        "red": "red"
    }),
    s=12,
    alpha=0.8
)

plt.xlabel("Actual Price (THB)")
plt.ylabel("Predicted Price (THB)")
plt.title("Actual vs Predicted (Color = Error Category)")
plt.xlim(0, max_val)
plt.ylim(0, max_val)

# ===== ADD METRICS AS TEXT BOX =====
metrics_text = (
    f"MAE: {mae:,.0f} THB\n"
    f"MAPE: {mape:.2f}%\n"
    f"RMSLE: {rmsle:.4f}\n"
    f"Hit-rate (≤20% error): {hit_rate:.2f}%"
)

plt.text(
    0.02, 0.98,
    metrics_text,
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment='top',
    bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8)
)

plt.show()

In [ ]:
# Extract linear regression from pipeline
linreg = pipeline.named_steps["reg"]

# Extract feature names after transformation
feature_names = (
    pipeline.named_steps["preprocessor"]
    .get_feature_names_out()
)

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": linreg.coef_
}).sort_values(by="coefficient", key=abs, ascending=False)

coef_df.head(20)


In [ ]:
df.info()